<a href="https://colab.research.google.com/github/ramyabardae/Movie-review-classification/blob/main/Movie_review_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary classification of IMDB movie reviews using Tensorflow & NLTK


### NAME - RAMYA BARDAE PRN - 17070123080


# **IMPORTING DEPENDENCIES AND GETTING THE DATA**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re
from keras.utils import to_categorical
from keras import models
from keras import layers
from nltk.stem import SnowballStemmer
from keras.utils import to_categorical
from keras import models
from keras import layers

In [2]:
from google.colab import files
uploaded = files.upload()

Saving moviereviews.csv to moviereviews.csv


In [4]:
df = pd.read_csv('moviereviews.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
sentences = df['review']
labels = df['sentiment']

In [6]:
print(df.shape)
df.sentiment.value_counts()

(50000, 2)


negative    25000
positive    25000
Name: sentiment, dtype: int64

In [7]:
df.review[2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

## **PREPROCESSING THE DATA**

### 1) Remove HTML tags

In [8]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

df.review = df.review.apply(clean)
df.review[2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

### 2) Remove special characters

In [9]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

df.review = df.review.apply(is_special)
df.review[2]

'I thought this was a wonderful way to spend time on a too hot summer weekend  sitting in the air conditioned theater and watching a light hearted comedy  The plot is simplistic  but the dialogue is witty and the characters are likable  even the well bread suspected serial killer   While some may be disappointed when they realize this is not Match Point 2  Risk Addiction  I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love This was the most I d laughed at one of Woody s comedies in years  dare I say a decade    While I ve never been impressed with Scarlet Johanson  in this she managed to tone down her  sexy  image and jumped right into a average  but spirited young woman This may not be the crown jewel of his career  but it was wittier than  Devil Wears Prada  and more interesting than  Superman  a great comedy to go see with friends '

### 3)Convert all letters to lowercase

In [10]:
def to_lower(text):
    return text.lower()

df.review = df.review.apply(to_lower)
df.review[2]

'i thought this was a wonderful way to spend time on a too hot summer weekend  sitting in the air conditioned theater and watching a light hearted comedy  the plot is simplistic  but the dialogue is witty and the characters are likable  even the well bread suspected serial killer   while some may be disappointed when they realize this is not match point 2  risk addiction  i thought it was proof that woody allen is still fully in control of the style many of us have grown to love this was the most i d laughed at one of woody s comedies in years  dare i say a decade    while i ve never been impressed with scarlet johanson  in this she managed to tone down her  sexy  image and jumped right into a average  but spirited young woman this may not be the crown jewel of his career  but it was wittier than  devil wears prada  and more interesting than  superman  a great comedy to go see with friends '

### 4. Remove any stopwords such as "an", "if" to make a more accurate model

In [ ]:
!pip install -q wordcloud
import wordcloud
import nltk
nltk.download('stopwords')
nltk.download('wordnet')



In [13]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))


In [14]:
def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

df.review = df.review.apply(rem_stopwords)
df.review[2]

['thought',
 'wonderful',
 'way',
 'spend',
 'time',
 'hot',
 'summer',
 'weekend',
 'sitting',
 'air',
 'conditioned',
 'theater',
 'watching',
 'light',
 'hearted',
 'comedy',
 'plot',
 'simplistic',
 'dialogue',
 'witty',
 'characters',
 'likable',
 'even',
 'well',
 'bread',
 'suspected',
 'serial',
 'killer',
 'may',
 'disappointed',
 'realize',
 'match',
 'point',
 '2',
 'risk',
 'addiction',
 'thought',
 'proof',
 'woody',
 'allen',
 'still',
 'fully',
 'control',
 'style',
 'many',
 'us',
 'grown',
 'love',
 'laughed',
 'one',
 'woody',
 'comedies',
 'years',
 'dare',
 'say',
 'decade',
 'never',
 'impressed',
 'scarlet',
 'johanson',
 'managed',
 'tone',
 'sexy',
 'image',
 'jumped',
 'right',
 'average',
 'spirited',
 'young',
 'woman',
 'may',
 'crown',
 'jewel',
 'career',
 'wittier',
 'devil',
 'wears',
 'prada',
 'interesting',
 'superman',
 'great',
 'comedy',
 'go',
 'see',
 'friends']

### 5) Apply stemming to cut off the strting and the ending words

In [15]:
def stem_txt(text):
    ss = SnowballStemmer('english')
    return " ".join([ss.stem(w) for w in text])

df.review = df.review.apply(stem_txt)
df.review[0]

'one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

### To filter sentences, and check if all the sentences are filtered

In [16]:
filtered_sentences = []
for s in sentences:
    word_tokens = word_tokenize(s) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentences.append(" ".join(filtered_sentence))

In [17]:
print(len(filtered_sentences[2]))

521


In [19]:
len(filtered_sentences)

50000

### Replacing a 1 or 0 in the Sentiment column for classification

In [20]:
df.sentiment.replace('positive',1,inplace=True)
df.sentiment.replace('negative',0,inplace=True)
df.head(10)

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1
5,probabl time favorit movi stori selfless sacri...,1
6,sure would like see resurrect date seahunt ser...,1
7,show amaz fresh innov idea 70 first air first ...,0
8,encourag posit comment film look forward watch...,0
9,like origin gut wrench laughter like movi youn...,1


   ## BUILDING A MODEL

In [21]:
vocab_size = 10000
embedding_dim = 16
max_length = 500
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 25000

### Splitting into test and train data

In [52]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [54]:
training_padded = np.asarray(training_padded).astype(np.int)
training_labels = np.asarray(training_labels).astype(np.int)
testing_padded = np.asarray(testing_padded).astype(np.int)
testing_labels = np.asarray(testing_labels).astype(np.int)

### Splitting up a larger body of text into smaller lines using tokenizer and also performing padding for getting an accurate model

In [55]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [56]:
print(len(training_sequences))
print(training_padded)

25000
[[   4  234  362 ...    0    0    0]
 [ 111   53  210 ...    0    0    0]
 [ 101  111   33 ...    0    0    0]
 ...
 [2486 1041 2173 ...    0    0    0]
 [   1  376 7783 ...    0    0    0]
 [   3   49   35 ...    0    0    0]]


### Preparing a neural network

In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [45]:
# Loss function and optimizer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [57]:
# Fine tuning the model to test the performance of the trained model
num_epochs = 10
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/10
782/782 - 4s - loss: 0.5724 - accuracy: 0.7322 - val_loss: 0.3813 - val_accuracy: 0.8600
Epoch 2/10
782/782 - 3s - loss: 0.3022 - accuracy: 0.8875 - val_loss: 0.2931 - val_accuracy: 0.8837
Epoch 3/10
782/782 - 3s - loss: 0.2386 - accuracy: 0.9097 - val_loss: 0.3047 - val_accuracy: 0.8751
Epoch 4/10
782/782 - 3s - loss: 0.2085 - accuracy: 0.9210 - val_loss: 0.2842 - val_accuracy: 0.8875
Epoch 5/10
782/782 - 3s - loss: 0.1879 - accuracy: 0.9298 - val_loss: 0.2828 - val_accuracy: 0.8894
Epoch 6/10
782/782 - 3s - loss: 0.1694 - accuracy: 0.9375 - val_loss: 0.2920 - val_accuracy: 0.8868
Epoch 7/10
782/782 - 3s - loss: 0.1555 - accuracy: 0.9440 - val_loss: 0.2992 - val_accuracy: 0.8902
Epoch 8/10
782/782 - 3s - loss: 0.1435 - accuracy: 0.9479 - val_loss: 0.3524 - val_accuracy: 0.8766
Epoch 9/10
782/782 - 4s - loss: 0.1309 - accuracy: 0.9542 - val_loss: 0.3251 - val_accuracy: 0.8870
Epoch 10/10
782/782 - 4s - loss: 0.1206 - accuracy: 0.9575 - val_loss: 0.3444 - val_accuracy: 0.8842

## TESTING THE MODEL

In [58]:
def predict_sentence(sentence):
    sentence = [sentence]
    sequences = tokenizer.texts_to_sequences(sentence)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    sentiment = model.predict(padded)
    return ("positive",float(sentiment)) if sentiment > 0.5 else ("negative", float(sentiment))

In [59]:
df.review[4]

'petter mattei love time money visual stun film watch mr mattei offer us vivid portrait human relat movi seem tell us money power success peopl differ situat encount variat arthur schnitzler play theme director transfer action present time new york differ charact meet connect one connect one way anoth next person one seem know previous point contact stylish film sophist luxuri look taken see peopl live world live habitat thing one get soul pictur differ stage loneli one inhabit big citi exact best place human relat find sincer fulfil one discern case peopl encount act good mr mattei direct steve buscemi rosario dawson carol kane michael imperioli adrian grenier rest talent cast make charact come aliv wish mr mattei good luck await anxious next work'

In [72]:
predict_sentence(df.review[4])

('positive', 0.9998877048492432)

In [65]:
predict_sentence(df.review[5000])

('positive', 0.8377047777175903)

In [68]:
predict_sentence("movie was very very bad")

('negative', 0.4686238467693329)